In [5]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from tensorflow.keras import applications
import tensorflow as tf

In [6]:
train='D:/Final/Graduation project/datasets/casting_data/casting_data/train'
test='D:/Final/Graduation project/datasets/casting_data/casting_data/test'
class_names = ['def_front', 'ok_front']
folder_names = ['def_aug', 'ok_aug']
class_to_folder = dict(zip(range(len(class_names)), folder_names))
batch_size = 32
img_size = (224, 224)

In [7]:
trdata = ImageDataGenerator(rescale=1/255.0,
                            rotation_range=20,
                            zoom_range=0.05,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            horizontal_flip=True,
                            fill_mode="nearest",
                            validation_split=0.20)

train_generator = trdata.flow_from_directory(directory=train_dir,
                                              target_size=img_size,
                                              subset='training',
                                              batch_size=batch_size,
                                              shuffle=True,
                                              color_mode='rgb',
                                              class_mode='categorical',
                                              save_to_dir=train,
                                              classes=class_names)

tsdata = ImageDataGenerator(rescale=1/255.0)
test_generator = tsdata.flow_from_directory(directory=test_dir,
                                             target_size=img_size,
                                             color_mode='rgb',
                                             batch_size=batch_size,
                                             class_mode=None,
                                             shuffle=False,
                                             save_to_dir=test,
                                             classes=class_names)

valid_generator = trdata.flow_from_directory(directory=train,
                                              target_size=(224, 224),
                                              color_mode="rgb",
                                              batch_size=32 ,
                                              class_mode='categorical',
                                              subset='validation',
                                              shuffle=True,
                                              ).repeat()
#it will generate 30* 32 image
num_images_to_generate = 30
for i in range(num_images_to_generate):
    batch = train_generator.next()
    for j in range(batch_size):
        img = batch[0][j]
        label = batch[1][j]
        class_index = tf.argmax(label).numpy()
        folder_path = os.path.join(train, 'def_front' if label[0] == 1 else 'ok_front')
        save_path = os.path.join(folder_path, f'{class_to_folder[class_index]}_{i}.jpg')
        tf.keras.preprocessing.image.save_img(save_path, img)

for i in range(num_images_to_generate):
    batch = test_generator.next()
    for j in range(batch_size):
        img = batch[0][j]
        label = batch[1][j]
        class_index = tf.argmax(label).numpy()
        folder_path = os.path.join(test, 'def_front' if label[0] == 1 else 'ok_front')
        save_path = os.path.join(folder_path, f'{class_to_folder[class_index]}_{i}.jpg')
        tf.keras.preprocessing.image.save_img(save_path, img)


NameError: ignored

In [ ]:
VGG = keras.applications.vgg16.VGG16(input_shape=(224,224,3), include_top = False, weights='imagenet')

In [ ]:
VGG.trainable = False
#Not train the front 13 layers, train only last two layers

In [ ]:
model = keras.Sequential([
    VGG,
    keras.layers.Flatten(),
    keras.layers.Dense(units=256, activation="relu"),
    keras.layers.Dense(units=256, activation="relu"),
    keras.layers.Dense(units=2, activation="softmax"), 
])

In [ ]:
model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])
model.summary()

In [ ]:
hist = model.fit(traindata , steps_per_epoch = 195   , epochs = 5 , #traindata.n 
                           validation_data = valid_generator, validation_steps = 48)
model.save('vggclf_v5.h5')


# Your input ran out of data; interrupting training. Make sure that your 
#dataset or generator can generate at least `steps_per_epoch * epochs` batches 
# image data generator should generate at least 195*10 = 1950 so if i set batch size to 32 so i need a 
# 6230 / 32 = 195 batch to go through entire data set  

In [ ]:
model.evaluate(valid_generator,steps=48)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:
pred = model.predict(testdata,steps = testdata.n, verbose = 1)

pred

In [ ]:
from keras.preprocessing import image
img = keras.utils.load_img("D:/Final/Graduation project/datasets/12.png",target_size=(224,224))
img = np.asarray(img)


plt.imshow(img)
img = np.expand_dims(img, axis=0)

In [ ]:
from keras.models import load_model
saved_model = load_model("vggclf_v5.h5")
output = model.predict(img)
if output[0][0] > output[0][1]:
    print("defected")
    print(output)
else:
    print("good")
    print(output)

In [ ]:
import pickle
  
# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)
model_from_pickle = pickle.loads(saved_model)